<center><img src="../image/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 9.机器人球体追踪

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：8.机器人球体追踪.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：机器人球体追踪<br>

In [1]:
import cv2
import numpy as np
from LOBOROBOT import LOBOROBOT # 载入机器人库
from gpiozero import Button
from gpiozero import LED
import time  

In [2]:
kernel = np.ones((5,5),np.uint8)
# 按键及LED初始化
Btn  = Button(19,pull_up=True)   # 按键端口
Gpin    = LED(5)
Rpin    = LED(6)

In [3]:
clbrobot = LOBOROBOT()  # 实例化机器人对象

# Configure min and max servo pulse lengths
servo_min = 150  # Min pulse length out of 4096
servo_max = 600  # Max pulse length out of 4096

# 频率设置为50hz，适用于舵机系统。
clbrobot.set_servo_angle(10,90)  # 底座舵机 90 
clbrobot.set_servo_angle(9,30)  # 顶部舵机 30
time.sleep(0.5)

## 打开摄像头

In [4]:
import libcamera
from picamera2 import Picamera2

picamera = Picamera2()
config = picamera.create_preview_configuration(main={"format": 'RGB888', "size": (320, 240)},
                                               raw={"format": "SRGGB12", "size": (1920, 1080)})
config["transform"] = libcamera.Transform(hflip=0, vflip=1)
picamera.configure(config)
picamera.start()

[0:00:47.554335322] [2113]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:00:47.562916377] [2150]  INFO RPI pisp.cpp:653 libpisp version v1.0.2 fa44a258644a 22-11-2023 (21:59:22)
[0:00:47.572256618] [2150]  INFO RPI pisp.cpp:1112 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media0 and ISP device /dev/media2 using PiSP variant BCM2712_C0
[0:00:47.574078618] [2113]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:00:47.582799303] [2153]  INFO RPI pisp.cpp:653 libpisp version v1.0.2 fa44a258644a 22-11-2023 (21:59:22)
[0:00:47.592514507] [2153]  INFO RPI pisp.cpp:1112 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media0 and ISP device /dev/media2 using PiSP variant BCM2712_C0
[0:00:47.595024673] [2113]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format Y16 
[0:00:47.595070599] [2113]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format RGB6
[0:00:

In [5]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

FGmaskComp_img = widgets.Image(format='jpeg', width=320, height=240)
frame_img = widgets.Image(format='jpeg', width=320, height=240)

dispaly_img = widgets.HBox([FGmaskComp_img,frame_img])
display(dispaly_img)

In [6]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [7]:
hueLower = widgets.IntSlider(min=12,max=179,step=1,description='hueLower：',value=20)
hueUpper = widgets.IntSlider(min=30,max=179,step=1,description='hueUpper：',value=30)

satLow = widgets.IntSlider(min=50,max=255,step=1,description='satLow：',value=50)
satHigh = widgets.IntSlider(min=255,max=255,step=1,description='satHigh：',value=255)

valLow = widgets.IntSlider(min=50,max=255,step=1,description='valLow：',value=50)
valHigh = widgets.IntSlider(min=255,max=255,step=1,description='valHigh：',value=255)

slider_img=widgets.VBox([
              widgets.HBox([hueLower,hueUpper]),
              widgets.HBox([satLow,satHigh]),
              widgets.HBox([valLow,valHigh])
             ])
display(slider_img)

## 按键处理函数

In [8]:
# 按键标志位
keyflag = 0

# 按键控制函数
def keysacn():
    global keyflag
    print('*****************************************') 
    print('* makerobo Button Pressed!*') 
    print('*****************************************')
    Rpin.on()  # 打开红色LED
    Gpin.off() # 关闭绿色LED
    keyflag = 1   # 按键标志位置1

def released():
    print("button was released")
    Rpin.off()   # 关闭红色LED
    Gpin.on()    # 打开绿色LED

# 按键中断函数
Btn.when_pressed = keysacn
Btn.when_released = released

In [9]:
def Video_display():
    while(1):
        if keyflag == 1:
            frame = picamera.capture_array()
            frame=cv2.flip(frame, 1) 
            
            # 转换到HSV
            hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
            hue,sat,val = cv2.split(hsv)
            
            hueLow=hueLower.value
            hueUp=hueUpper.value
        
            Ls= satLow.value
            Us = satHigh.value
        
            Lv=valLow.value
            Uv=valHigh.value
        
            l_b=np.array([hueLow,Ls,Lv])
            u_b=np.array([hueUp,Us,Uv])
        
        
            FGmaskComp=cv2.inRange(hsv,l_b,u_b)
            FGmaskComp_img.value = bgr8_to_jpeg(FGmaskComp)
            
            # 应用阈值
            hthresh = cv2.inRange(np.array(hue),np.array(hueLow),np.array(hueUp))
            sthresh = cv2.inRange(np.array(sat),np.array(Ls),np.array(Us))
            vthresh = cv2.inRange(np.array(val),np.array(Lv),np.array(Uv))
        
            # h s和v
            tracking = cv2.bitwise_and(hthresh,cv2.bitwise_and(sthresh,vthresh))
        
            # 一些morpholigical过滤
            dilation = cv2.dilate(tracking,kernel,iterations = 1)
            closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel)
            closing = cv2.GaussianBlur(closing,(5,5),0)
        
            # 使用HoughCircles检测圆
            circles = cv2.HoughCircles(closing,cv2.HOUGH_GRADIENT,2,120,param1=120,param2=50,minRadius=10,maxRadius=0)
            # circles = np.uint16(np.around(circles))
            clbrobot.t_stop(0)
        
            # 画圆圈
            if circles is not None:
                x, y, r = circles[0][0]
                x_p = int(round(x))
                print(x_p)
                for i in circles[0,:]:
                        # 如果球很远，用绿色画出来
                        if int(round(i[2])) < 30:
                                cv2.circle(frame,(int(round(i[0])),int(round(i[1]))),int(round(i[2])),(0,255,0),5)
                                cv2.circle(frame,(int(round(i[0])),int(round(i[1]))),2,(0,255,0),10)
                        # 或者用红色画
                        elif int(round(i[2])) > 35:
                                cv2.circle(frame,(int(round(i[0])),int(round(i[1]))),int(round(i[2])),(0,0,255),5)
                                cv2.circle(frame,(int(round(i[0])),int(round(i[1]))),2,(0,0,255),10)
    
                # 设定一个范围
                x_Lower = 150
                x_Upper = 180
                # 判断X方向范围来判断机器人的运动
                if x_p > x_Lower and x_p < x_Upper:
                        clbrobot.t_up(30,0)
                elif x_p < x_Lower:
                    clbrobot.turnLeft(20,0)              
                elif x_p > x_Upper:
                    clbrobot.turnRight(20,0)    
                else:
                    clbrobot.t_stop(0)
            # 在帧中显示结果
            frame_img.value = bgr8_to_jpeg(frame)

## 打开线程

In [10]:
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

/tmp/ipykernel_2113/3481196328.py:2: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  t.setDaemon(True)


button was released
*****************************************
* makerobo Button Pressed!*
*****************************************
147
149
137
135
145
149
191
149
149
145
147
147
181
149
135
153
135
151
149
135
141
145
143
133
145
147
145
149
149
147
149
143
147
133
143
147
147
147
147
149
137
149
133
145
133
145
147
143
149
149
147
145
147
135
147
143
149
135
149
143
145
181
153
133
147
135
147
143
149
149
135
135
147
135
143
145
147
143
147
181
149
143
133
133
107
149
147
149
149
157
149
145
149
147
143
147
149
145
147
147
143
143
143
133
147
143
143
151
135
149
133
143
149
143
133
143
143
133
147
149
149
149
147
147
143
143
175
161
137
55
123
161
125
131
123
121
115
107
129
163
169
167
175
173
173
161
163
173
163
159
183
145
149
159
171
163
155
159
161
143
147
153
151
159
161
159
161
151
147
139
141
139
141
137
141
139
151
143
145
141
145
143
139
143
149
145
149
149
151
141
145
149
147
147
151
147
143
161
153
153
145
153
147
141
153
153
157
167
153
153
151
153
153
155
143
137
139
1

## 关闭线程

In [11]:
# 结束线程
clbrobot.t_stop(0) # 停止电机
stop_thread(t)

129
